<a href="https://colab.research.google.com/github/matlogica/AADC-Python/blob/main/getting-started/01-hello-world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AADC basic "Hello World" example
This example demonstrates how to use the AADC library to record a simple function and calculate its derivatives

In [20]:
import sys

### Please uncomment next line if you don't have AADC installed locally

In [21]:
#pip install https://matlogica.com/DemoReleases/aadc-1.7.5.27-cp3{sys.version_info.minor}-cp3{sys.version_info.minor}-linux_x86_64.whl

In [22]:
import aadc
import numpy as np

# Simple function to record
The function can take arbitrary input types and can be defined in external file

In [23]:
def F(x,y,z):
    return np.exp(x / y + z)

# Object to hold recorded AADC Kernels
allow to speed up the calculation of the function itself and AAD for its derivatives


In [24]:
funcs = aadc.Functions()

# Define function arguments.
Note you need to use aadc.idouble instead of float

In [25]:
x = aadc.idouble(1.0)
y = aadc.idouble(2.0)
z = aadc.idouble(3.0)

# Trigger recording of the function

In [26]:
funcs.start_recording()

You are using evaluation version of AADC. Expire date is 20240901


# Mark the function arguments as input and save reference argument ids

In [27]:
xArg = x.mark_as_input()
yArg = y.mark_as_input()
zArg = z.mark_as_input()

# Call the function and add some operations

In [28]:
f = F(x,y,z) + x

# Mark the result as output

In [29]:
fRes = f.mark_as_output()

# Stop recording
After this step AADC Kernel containing native machine CPU code will be generated and stored in the funcs object

In [30]:
funcs.stop_recording()

# Check if the function is recorded properly
So it can be used for arbitrary input values. This should return 0 if everything is OK, indicating that no branches in the function are not supported.


In [31]:
funcs.print_passive_extract_locations()

Number active to passive conversions: 

'Number active to passive conversions: 0 while recording Python\n'

0 while recording Python


# New samples of input values
To calculate the function and its derivatives at. Note that the x input is a vector of 20 samples and the y and z are scalars

In [32]:
inputs = {
    xArg:(1.0 * np.random.normal(1, 0.2, 20)),
    yArg:(2.0),
    zArg:(3.0),
}

Key: what output, value: what gradients are needed

In [33]:
request = {fRes:[xArg,yArg,zArg]}  

Run AADC Kernel for array inputs using 4 CPU threads and avx2

In [34]:
Res = aadc.evaluate(funcs, request, inputs, aadc.ThreadPool(4))

## Function output

In [38]:
Res[0][fRes]

array([32.10395476, 34.83481649, 40.6722964 , 41.2480603 , 35.94731013,
       39.0797347 , 35.44086099, 35.21160137, 32.0147729 , 30.89374544,
       32.75031696, 35.24848732, 38.24760169, 40.82728994, 29.13668977,
       31.65707208, 35.65670017, 41.72080241, 34.09176272, 28.74009506])

## df/dx

In [36]:
Res[1][fRes][xArg]

array([16.61086317, 17.89711799, 20.66420123, 20.93824331, 18.42273209,
       19.9071785 , 18.18334284, 18.07503599, 16.56896007, 16.04282444,
       16.91476406, 18.0924591 , 19.51222248, 20.73795439, 15.22051226,
       16.40095908, 18.28534394, 21.16338566, 17.54655619, 15.03532312])

## df/dy

In [37]:
Res[1][fRes][yArg]


array([ -6.88617357,  -8.79140582, -13.21330038, -13.67338492,
        -9.59858341, -11.96236094,  -9.22896132,  -9.06282625,
        -6.82584271,  -6.0780273 ,  -7.32706851,  -9.08950582,
       -11.32167483, -13.33674988,  -4.94635815,  -6.58509535,
        -9.38604809, -14.05419332,  -8.26211207,  -4.69796528])